In [1]:
import os
import pickle
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import custom_object_scope


def load_metadata(directory):
    metadata = []
    for file_name in os.listdir(directory):
        if file_name.endswith('meta.pkl'):
            with open(os.path.join(directory, file_name), 'rb') as f:
                data = pickle.load(f)
                metadata.append({
                    'file_name': file_name,
                    'hyperparameter': data['hyperparameter'],
                    'profit_1': data['profit_1'],
                    'profit_2': data['profit_2'],
                    'elapsed_time': data['elapsed_time'],
                    'memory': data['memory']
                })
    return metadata

# Directory where the test files are located
directory_path = 'C:/Users/lanza/Master_Thesis_EL/Integrated-vs-Seperated-Master-Thesis/test/'

# Load metadata from all models
metadata_list = load_metadata(directory_path)

# Convert to DataFrame for better visualization
df_metadata = pd.DataFrame(metadata_list)



C:\Users\lanza\AppData\Local\Temp\ipykernel_32680\1529656650.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
print(df_metadata.head())

                         file_name  \
0  test_1_ANN_complex_IOA_meta.pkl   
1  test_1_ANN_complex_SOA_meta.pkl   
2   test_1_ANN_simple_IOA_meta.pkl   
3   test_1_ANN_simple_SOA_meta.pkl   
4  test_1_XGB_complex_IOA_meta.pkl   

                                      hyperparameter    profit_1    profit_2  \
0     [2, 14, 0.02612361989816738, 18, 10, 16, relu] -410.811173         NaN   
1   [10, 15, 0.043855260179841815, 28, 10, 16, tanh]   93.699455   93.741349   
2     [9, 19, 0.08574228345289107, 16, 10, 16, relu]  430.362416         NaN   
3  [7, 14, 0.0017167469895068748, 25, 10, 128, tanh] -284.019890  416.263971   
4  {'learning_rate': 0.14210353931582873, 'max_de...   99.287182         NaN   

   elapsed_time              memory  
0      7.223674  (6022104, 6127568)  
1     33.704140  (3758361, 6902549)  
2      9.102684  (9735510, 9766793)  
3      9.992877  (3091060, 6817707)  
4      1.160164     (45261, 217373)  


In [27]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

def is_feasible(q, d, alpha, u, o, M):
    """
    Check if the point q satisfies the constraints in the MILP.
    """
    n_prods = q.shape[0]
    hist = d.shape[0]
    
    for i in range(n_prods):
        for t in range(hist):
            y_t_i = q[i] - d[t, i] - np.sum(alpha[:, i] * np.maximum(d[t, :] - q, 0))
            v_t_i = np.maximum(d[t, i] - q[i], 0)
            z_t_i = (v_t_i > 0).astype(int)
            
            # Constraints
            if not (y_t_i >= 0):
                return False
            if not (v_t_i <= d[t, i] - q[i] + M[i] * z_t_i):
                print(2)
                return False
            if not (v_t_i >= d[t, i] - q[i] - M[i] * z_t_i):
                print(3)
                return False
            if not (v_t_i <= d[t, i] * (1 - z_t_i)):
                return False
    return True

def monte_carlo_feasible_region_size(d, alpha, u, o, lower_bounds, upper_bounds, num_samples=10000):
    """
    Estimate the size of the feasible region using Monte Carlo sampling
    
    Parameters:
    d (numpy.ndarray): Demand samples of shape (n, N_PRODUCTS)
    alpha (numpy.ndarray): Substitution rates, shape (N_PRODUCTS, N_PRODUCTS)
    u (numpy.ndarray): Underage costs, shape (1, N_PRODUCTS)
    o (numpy.ndarray): Overage costs, shape (1, N_PRODUCTS)
    lower_bounds (numpy.ndarray): Lower bounds of the order quantities
    upper_bounds (numpy.ndarray): Upper bounds of the order quantities
    num_samples (int): Number of random samples to generate
    
    Returns:
    float: Estimated size of the feasible region
    """
    n_prods = d.shape[1]
    M = (np.max(d + np.matmul(d, alpha), axis=0) - np.min(d, axis=0))*1000
    feasible_count = 0
    
    for _ in range(num_samples):
        # Generate a random point within the variable bounds
        q = np.random.randint(lower_bounds, upper_bounds + 1, n_prods)
        
        # Check if the point is feasible
        if is_feasible(q, d, alpha, u, o, M):
            
            feasible_count += 1
    
    # Debug statements
    print(f"Total number of samples: {num_samples}")
    print(f"Number of feasible samples: {feasible_count}")
    
    # Estimate the proportion of feasible points
    proportion_feasible = feasible_count / num_samples
    
    # Debug statement
    print(f"Proportion of feasible points: {proportion_feasible}")
    
    # Estimate the size of the feasible region
    total_region_size = np.prod(upper_bounds - lower_bounds + 1)
    feasible_region_size = proportion_feasible * total_region_size
    
    return feasible_region_size

# Example parameters
d = np.array([[20, 30, 25], [22, 31, 24], [19, 29, 27]])  # Example demand samples for 3 products
alpha = np.array([[0.0, 0.1, 0.2], [0.1, 0.0, 0.3], [0.2, 0.3, 0.0]])  # Example substitution rates
underage = np.array([[10, 8, 9]])  # Example underage costs
overage = np.array([[5, 4, 6]])  # Example overage costs

lower_bound_value = 0  # Integer lower bound
upper_bound_value = 50  # Integer upper bound

n_prods = d.shape[1]
lower_bounds = np.full(n_prods, lower_bound_value)
upper_bounds = np.full(n_prods, upper_bound_value)

# Estimate the feasible region size
estimated_feasible_region_size = monte_carlo_feasible_region_size(d, alpha, underage, overage, lower_bounds, upper_bounds)
print(f"Estimated feasible region size: {estimated_feasible_region_size}")


TypeError: 'int' object is not subscriptable